# Melanoma Dermoscopic Prognosis - Colab Training Notebook

This notebook sets up the environment, downloads data from Google Drive, and trains the model using `main.py`.

**Note:** The model uses image-only input (dermoscopic images). Clinical features are not required.


## Step 1: Clone Repository


In [ ]:
import os
import sys
from pathlib import Path

# Clone the repository
repo_url = "https://github.com/Salahuddin-quadri/Melanoma-Dermoscopic-Prognosis.git"
repo_name = "Melanoma-Dermoscopic-Prognosis"

# Check if we're already in the repo directory
if os.path.exists("src/main.py"):
    print("Already in repository directory. Skipping clone.")
    print(f"Current directory: {os.getcwd()}")
elif os.path.exists(repo_name):
    print(f"Repository {repo_name} already exists. Changing to it.")
    os.chdir(repo_name)
    print(f"Current directory: {os.getcwd()}")
else:
    # Clone the repository
    get_ipython().system(f'git clone {repo_url}')
    os.chdir(repo_name)
    print(f"Cloned and changed to: {os.getcwd()}")

# Verify we're in the right place
assert os.path.exists("src/main.py"), "src/main.py not found! Check repository structure."
print("✓ Repository setup complete!")


## Step 2: Install Dependencies


In [ ]:
# Install PyTorch with CUDA support
!pip install torch==2.3.1 torchvision==0.18.1 --index-url https://download.pytorch.org/whl/cu118

# Install other dependencies
!pip install numpy==1.26.4 pandas==2.2.2 scikit-learn==1.4.2 scipy==1.11.4
!pip install matplotlib==3.8.4 seaborn==0.13.2 opencv-python==4.10.0.84 Pillow==10.4.0
!pip install tqdm==4.66.4 ipywidgets==8.1.3 imbalanced-learn==0.12.3
!pip install gdown

print("✓ Dependencies installed!")


## Step 3: Mount Google Drive and Download Data

**Important:** The data files are large (~1GB). We'll mount Google Drive and download directly from the shared folders.


In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

print("✓ Google Drive mounted successfully!")
print("Your Drive is now accessible at /content/drive/MyDrive")


### Download Data Folder from Google Drive

The data folder contains:
- `images/` - dermoscopic images
- `merged_dataset.csv` - metadata CSV
- `meta_data.csv` - additional metadata


In [ ]:
# Download the file from Google Drive
file_id = "1LCbiPcQXJperjrOhG4Xb947rkCiJPDRh"
output_filename = "downloaded_file.zip"

!gdown {file_id} -O {output_filename}

# Unzip only the 'data/' folder into the current directory
import zipfile
import os

with zipfile.ZipFile(output_filename, 'r') as zip_ref:
    members = [m for m in zip_ref.namelist() if m.startswith("data/")]
    zip_ref.extractall(members=members)

print("✓ 'data/' folder extracted to current directory.")
print("\nExtracted contents of 'data/':")
!ls -la data/

### Download DINO v3 Pretrained Model from Google Drive

The dino_v3 folder contains the pretrained checkpoint needed for training.


In [ ]:
# Google Drive folder ID for dino_v3 model
dino_folder_id = "1wlozH7SchoPkAgsDpn-DyBN-F2Ea8hmP"

# Create dino_v3 directory
dino_dest = "dino_v3"
os.makedirs(dino_dest, exist_ok=True)

print(f"Downloading dino_v3 folder (ID: {dino_folder_id})...")
print("This may take a while as the folder is large...")

# Use gdown to download the entire folder
get_ipython().system(f'gdown --folder https://drive.google.com/drive/folders/{dino_folder_id} -O {dino_dest} --remaining-ok')

print("\n✓ DINO v3 model download complete!")


### Verify Downloaded Files


In [ ]:
# Verify data files
data_dir = Path("data")
print("=" * 60)
print("DATA FOLDER VERIFICATION")
print("=" * 60)

if data_dir.exists():
    # Calculate total size
    total_size_mb = sum(f.stat().st_size for f in data_dir.rglob('*') if f.is_file()) / (1024 * 1024)
    print(f"Total size: {total_size_mb:.2f} MB")
    print(f"\nContents:")

    # Check for required files
    required_files = ["merged_dataset.csv", "meta_data.csv"]
    for file in required_files:
        file_path = data_dir / file
        if file_path.exists():
            size_mb = file_path.stat().st_size / (1024 * 1024)
            print(f"  ✓ {file} ({size_mb:.2f} MB)")
        else:
            print(f"  ✗ Missing: {file}")

    # Check for images folder
    images_dir = data_dir / "images"
    if images_dir.exists():
        num_images = len([f for f in images_dir.rglob("*") if f.is_file()])
        images_size_mb = sum(f.stat().st_size for f in images_dir.rglob("*") if f.is_file()) / (1024 * 1024)
        print(f"  ✓ images/ folder ({num_images} files, {images_size_mb:.2f} MB)")
    else:
        print(f"  ✗ Missing: images/ folder")
else:
    print("✗ Data directory not found!")

print("\n" + "=" * 60)
print("DINO_V3 FOLDER VERIFICATION")
print("=" * 60)

# Verify dino_v3 files
dino_dir = Path("dino_v3")
if dino_dir.exists():
    total_size_mb = sum(f.stat().st_size for f in dino_dir.rglob('*') if f.is_file()) / (1024 * 1024)
    print(f"Total size: {total_size_mb:.2f} MB")
    print(f"\nContents:")

    # Check for checkpoint
    checkpoint_path = dino_dir / "outputs_dino" / "checkpoints" / "best.pt"
    if checkpoint_path.exists():
        size_mb = checkpoint_path.stat().st_size / (1024 * 1024)
        print(f"  ✓ {checkpoint_path.relative_to(dino_dir)} ({size_mb:.2f} MB)")
    else:
        # Try alternative paths
        checkpoints = list(dino_dir.rglob("*.pt"))
        if checkpoints:
            for ckpt in checkpoints:
                size_mb = ckpt.stat().st_size / (1024 * 1024)
                print(f"  ✓ {ckpt.relative_to(dino_dir)} ({size_mb:.2f} MB)")
        else:
            print(f"  ✗ No checkpoint files (.pt) found")

    # List all files
    print(f"\nAll files in dino_v3:")
    for item in sorted(dino_dir.rglob("*")):
        if item.is_file():
            size_mb = item.stat().st_size / (1024 * 1024)
            print(f"  {item.relative_to(dino_dir)} ({size_mb:.2f} MB)")
else:
    print("✗ dino_v3 directory not found!")


## Step 4: Verify Setup


In [ ]:
# Check Python version
import sys
print(f"Python version: {sys.version}")

# Check PyTorch
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

# Check repository structure
print("\nRepository structure:")
print(f"  Current directory: {os.getcwd()}")
print(f"  src/main.py exists: {os.path.exists('src/main.py')}")
print(f"  requirements.txt exists: {os.path.exists('requirements.txt')}")

print("\n✓ Setup verification complete!")


## Step 5: Training Configuration

Configure your training parameters here. Adjust as needed.

**Model Architecture:** The model takes only dermoscopic images as input (no clinical features).
- DINO model: Uses Vision Transformer (ViT) backbone with domain-specific pretraining
- ResNet model: Uses ResNet50 backbone with ImageNet pretraining


In [ ]:
# Training configuration
# Note: Model takes only images as input (no clinical features)
config = {
    "metadata_path": "data/meta_data.csv",
    "image_dir": "data/images",
    "mode": "train",
    "model_type": "dino",  # "dino" or "resnet"
    "epochs": 200,
    "batch_size": 32,
    "image_size": [224,224],
    "output_dir": "outputs",
    "task": "classification",  # "classification" or "regression" (used when multitask=False)
    "multitask": True,  # Set to True for dual-head (classification + regression)
    "loss_alpha": 0.5,  # Weight for classification loss in multitask (0-1)
    "cls_loss_type": "weighted_bce",  # "bce", "weighted_bce", or "focal"
    "focal_gamma": 2.0,  # For focal loss (used when cls_loss_type="focal")
    "freeze_backbone_layers": 7,  # Number of ViT layers to freeze (0 = all trainable)
    "val_size": 0.15,
    "test_size": 0.15,
    "device": "auto",  # "cuda", "cpu", or "auto"
}

# Set DINO checkpoint path
# Try to find the checkpoint automatically
dino_checkpoint_candidates = [
    "dino_v3/outputs_dino/checkpoints/best.pt",
    "dino_v3/checkpoints/best.pt",
]

dino_checkpoint = None
for candidate in dino_checkpoint_candidates:
    if os.path.exists(candidate):
        dino_checkpoint = candidate
        break

if dino_checkpoint:
    config["dino_checkpoint"] = dino_checkpoint
    print(f"✓ Found DINO checkpoint: {dino_checkpoint}")
else:
    print("⚠ No DINO checkpoint found. Will use ImageNet pretrained weights.")
    # Don't set dino_checkpoint if not found

print("\nTraining configuration:")
for key, value in config.items():
    print(f"  {key}: {value}")


## Step 6: Run Training


In [ ]:
# Build command arguments
args_list = []
for key, value in config.items():
    if value is not None and value != "":
        if isinstance(value, bool):
            if value:
                args_list.append(f"--{key}")
        elif isinstance(value, list):
            args_list.append(f"--{key}")
            args_list.extend([str(v) for v in value])
        else:
            args_list.append(f"--{key}")
            args_list.append(str(value))

# Convert to string
args_str = " ".join(args_list)

print("=" * 60)
print("TRAINING COMMAND")
print("=" * 60)
print(f"python -m src.main {args_str}")
print("=" * 60)
print("\nStarting training...\n")


In [ ]:
# Run training
get_ipython().system(f'python -m src.main {args_str}')


In [ ]:
# List output files
# Note: Training creates subdirectories train1, train2, etc. (YOLO-style organization)
output_dir = Path(config["output_dir"])
if output_dir.exists():
    print(f"Output directory: {output_dir}")
    print("\nContents:")

    # Find training run directories (train1, train2, etc.)
    train_dirs = sorted([d for d in output_dir.iterdir() if d.is_dir() and d.name.startswith("train")])

    if train_dirs:
        latest_train_dir = train_dirs[-1]
        print(f"\nLatest training run: {latest_train_dir.name}")

        # List files in latest training run
        for item in latest_train_dir.rglob("*"):
            if item.is_file():
                size = item.stat().st_size / (1024 * 1024)  # Size in MB
                print(f"  {item.relative_to(output_dir)} ({size:.2f} MB)")

        # Check for checkpoints in latest training run
        checkpoint_dir = latest_train_dir / "checkpoints"
        if checkpoint_dir.exists():
            print(f"\nCheckpoints in {latest_train_dir.name}:")
            for ckpt in checkpoint_dir.glob("*.pt"):
                size = ckpt.stat().st_size / (1024 * 1024)
                print(f"  ✓ {ckpt.name} ({size:.2f} MB)")
        else:
            print(f"\n⚠ No checkpoints directory found in {latest_train_dir.name}")
    else:
        # Fallback: list all files directly
        for item in output_dir.rglob("*"):
            if item.is_file():
                size = item.stat().st_size / (1024 * 1024)  # Size in MB
                print(f"  {item.relative_to(output_dir)} ({size:.2f} MB)")
else:
    print(f"⚠ Output directory {output_dir} not found.")


In [7]:
# Diagnostic & robust import helper for your repo
import os, sys, importlib, traceback
from pathlib import Path
PROJECT_DIR = "/content/Melanoma-Dermoscopic-Prognosis"   # adjust only if you cloned elsewhere
print("PROJECT_DIR exists:", os.path.exists(PROJECT_DIR))
print("CWD:", os.getcwd())

# show top-level contents
print("\nTop-level files/folders in PROJECT_DIR:")
for p in sorted(os.listdir(PROJECT_DIR)):
    print(" -", p)

# make sure PROJECT_DIR is on sys.path
if PROJECT_DIR not in sys.path:
    sys.path.insert(0, PROJECT_DIR)
    print("\nInserted PROJECT_DIR into sys.path")

print("\nsys.path head:")
for i, p in enumerate(sys.path[:6]):
    print(f" [{i}] {p}")

# show src folder contents if present
src_dir = Path(PROJECT_DIR) / "src"
print("\nsrc exists:", src_dir.exists())
if src_dir.exists():
    print("src contents:")
    for p in sorted(os.listdir(src_dir)):
        print("  -", p)
    # if models is a directory, show its files too
    models_dir = src_dir / "models"
    print("src/models exists:", models_dir.exists())
    if models_dir.exists():
        print("src/models contents:")
        for p in sorted(os.listdir(models_dir)):
            print("    -", p)

# Try the normal import
print("\nAttempting: from src.models import create_dino_hybrid_model")
try:
    from src.models import create_dino_hybrid_model
    print("✅ Import succeeded. create_dino_hybrid_model:", create_dino_hybrid_model)
except Exception as e:
    print("❌ Import failed. Traceback:\n")
    traceback.print_exc()

    # Fallback: attempt to locate the file that defines the function and import it directly
    # Common candidate files:
    candidates = [
        src_dir / "models" / "__init__.py",
        src_dir / "models" / "dino_hybrid.py",
        src_dir / "models" / "factory.py",
        src_dir / "models.py"
    ]
    print("\nLooking for candidate files to import directly:")
    for c in candidates:
        print(" -", c, "exists:", c.exists())

    # Try to find any .py under src/models that contains the symbol name
    found_path = None
    if (src_dir / "models").exists():
        for p in (src_dir / "models").rglob("*.py"):
            try:
                txt = p.read_text(errors="ignore")
                if "create_dino_hybrid_model" in txt or "class DinoHybrid" in txt:
                    found_path = p
                    break
            except Exception:
                pass

    if found_path is None:
        # also try src root
        for p in src_dir.rglob("*.py"):
            try:
                txt = p.read_text(errors="ignore")
                if "create_dino_hybrid_model" in txt or "class DinoHybrid" in txt:
                    found_path = p
                    break
            except Exception:
                pass

    if found_path:
        print("\nFound candidate file defining the model function/class:", found_path)
        # Import directly from path
        try:
            import importlib.util
            spec = importlib.util.spec_from_file_location("custom_models_module", str(found_path))
            mod = importlib.util.module_from_spec(spec)
            spec.loader.exec_module(mod)
            # try to get function
            if hasattr(mod, "create_dino_hybrid_model"):
                create_dino_hybrid_model = getattr(mod, "create_dino_hybrid_model")
                print("✅ Loaded create_dino_hybrid_model from file:", found_path)
            else:
                print("⚠ Found file but it does not contain create_dino_hybrid_model. Available symbols:", [n for n in dir(mod) if not n.startswith("_")][:50])
        except Exception:
            print("❌ Failed to import directly from file. Traceback:")
            traceback.print_exc()
    else:
        print("\nCould not find a candidate file that defines create_dino_hybrid_model automatically.")
        print("Please check where the function is defined in your repo and run an import or share the listing above.")

print("\nIf import still fails, paste the output of this cell here and I'll give the exact next command.")


PROJECT_DIR exists: True
CWD: /content/Melanoma-Dermoscopic-Prognosis

Top-level files/folders in PROJECT_DIR:
 - .git
 - .gitattributes
 - .gitignore
 - CLASS_IMBALANCE_SOLUTIONS.md
 - LICENSE
 - README.md
 - data
 - dino_v3
 - error.txt
 - notebooks
 - plot_training_log.py
 - project_paper_context.txt
 - related_work_raw_notes.txt
 - requirements.txt
 - src
 - training.md

sys.path head:
 [0] /content/Melanoma-Dermoscopic-Prognosis
 [1] /content
 [2] /env/python
 [3] /usr/lib/python312.zip
 [4] /usr/lib/python3.12
 [5] /usr/lib/python3.12/lib-dynload

src exists: True
src contents:
  - __init__.py
  - evaluate.py
  - main.py
  - models
  - train.py
  - utils
src/models exists: True
src/models contents:
    - __init__.py
    - dino_hybrid.py
    - fusion.py
    - resnet50_hybrid.py

Attempting: from src.models import create_dino_hybrid_model
❌ Import failed. Traceback:


Looking for candidate files to import directly:
 - /content/Melanoma-Dermoscopic-Prognosis/src/models/__init__.py e

Traceback (most recent call last):
  File "/tmp/ipython-input-2528247755.py", line 40, in <cell line: 0>
    from src.models import create_dino_hybrid_model
ModuleNotFoundError: No module named 'src'


✅ Loaded create_dino_hybrid_model from file: /content/Melanoma-Dermoscopic-Prognosis/src/models/dino_hybrid.py

If import still fails, paste the output of this cell here and I'll give the exact next command.


In [8]:
# === Single-run cell: robust import + extract layers 13,14,15 + show overlays ===
import os, sys, math, json, traceback
from pathlib import Path
import numpy as np
import torch, torch.nn.functional as F
import matplotlib.pyplot as plt
from PIL import Image, ImageOps, ImageEnhance, ImageDraw, ImageFont

PROJECT_DIR = "/content/Melanoma-Dermoscopic-Prognosis"
OUT_DIR = Path("/content/feature_outputs")
ATT_DIR = OUT_DIR / "attention_fixed"
OVER_DIR = OUT_DIR / "overlays"
OUT_DIR.mkdir(parents=True, exist_ok=True)
ATT_DIR.mkdir(parents=True, exist_ok=True)
OVER_DIR.mkdir(parents=True, exist_ok=True)

# Replace this with a real dermoscopic image path you uploaded to Colab
IMAGE_PATH = "/content/sample_lesion.png"   # <-- change to your uploaded image path

# Ensure repo on sys.path
if PROJECT_DIR not in sys.path:
    sys.path.insert(0, PROJECT_DIR)

# Try normal import, else fallback to direct file import
create_dino_hybrid_model = None
try:
    from src.models import create_dino_hybrid_model as _factory
    create_dino_hybrid_model = _factory
    print("Imported create_dino_hybrid_model via package import.")
except Exception as e:
    print("Package import failed. Falling back to loading from file...")
    # Path we discovered earlier
    fallback_path = Path(PROJECT_DIR) / "src" / "models" / "dino_hybrid.py"
    print("Attempting to load from:", fallback_path)
    import importlib.util
    spec = importlib.util.spec_from_file_location("dino_hybrid_mod", str(fallback_path))
    mod = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(mod)
    # The factory may be defined at module scope or returned by a helper — try to get it
    if hasattr(mod, "create_dino_hybrid_model"):
        create_dino_hybrid_model = getattr(mod, "create_dino_hybrid_model")
        print("Loaded create_dino_hybrid_model from file.")
    else:
        # if not present, show available names for debugging
        print("Fallback file loaded but factory not found. Available symbols:", [n for n in dir(mod) if not n.startswith("_")][:200])
        raise RuntimeError("create_dino_hybrid_model not found in fallback module.")

# Create the model using the correct signature
model = create_dino_hybrid_model(
    task="classification",
    multitask=False,
    arch="vit_b_16",
    pretrained=False,
    dino_checkpoint=None,
    use_tokens=False,
    hidden_dim=256,
    dropout=0.1,
    freeze_backbone_layers=0
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device).eval()
print("Model ready on", device)

# Locate backbone and encoder layers
backbone = None
if hasattr(model, "backbone") and hasattr(model.backbone, "vit"):
    backbone = model.backbone.vit
elif hasattr(model, "vit"):
    backbone = model.vit
elif hasattr(model, "backbone") and hasattr(model.backbone, "vit"):
    backbone = model.backbone.vit

if backbone is None:
    raise RuntimeError("Backbone not found in model.")

# find encoder layers container
if hasattr(backbone, "encoder") and hasattr(backbone.encoder, "layers"):
    encoder_layers = backbone.encoder.layers
elif hasattr(backbone, "blocks"):
    encoder_layers = backbone.blocks
else:
    encoder_layers = None
    for n in dir(backbone):
        a = getattr(backbone, n)
        if isinstance(a, torch.nn.ModuleList):
            encoder_layers = a
            break
if encoder_layers is None:
    raise RuntimeError("Could not locate encoder layers container.")

num_layers = len(encoder_layers)
print("Encoder layers detected:", num_layers)

# request layers 13,14,15
requested = [13,14,15]
selected = [i for i in requested if i < num_layers]
if not selected:
    raise RuntimeError(f"Model has {num_layers} layers; none of {requested} are valid. Choose indices < {num_layers}.")
print("Using layers:", selected)

# Freeze only selected layers
for idx in selected:
    for p in encoder_layers[idx].parameters():
        p.requires_grad = False
print("Frozen layers:", selected)

# Capture inputs to these encoder blocks
inputs_store = {}
hooks = []
def make_hook(i):
    def hook(module, inp, out):
        try:
            inputs_store[f"layer_{i}"] = inp[0].detach().cpu()
        except Exception as exc:
            print("Hook store failed for", i, exc)
    return hook

for idx in selected:
    hooks.append(encoder_layers[idx].register_forward_hook(make_hook(idx)))
    print("Attached hook to layer", idx)

# Prepare input
from torchvision import transforms
if os.path.exists(IMAGE_PATH):
    pil = Image.open(IMAGE_PATH).convert("RGB")
    tf = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor(),
                             transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])])
    img = tf(pil).unsqueeze(0).to(device)
    print("Using real image:", IMAGE_PATH)
else:
    print("IMAGE_PATH not found; using random dummy tensor (results will be meaningless).")
    img = torch.randn(1,3,224,224).to(device)

# Forward pass
try:
    _ = model(img)
    print("Forward pass complete; captured keys:", list(inputs_store.keys()))
except Exception as e:
    # try backbone forward_features as fallback
    try:
        if hasattr(backbone, "forward_features"):
            _ = backbone.forward_features(img)
            print("Backbone forward_features executed.")
    except Exception as e2:
        print("Forward failed:", e, e2)

# remove hooks
for h in hooks:
    h.remove()

# Compute attention from Q/K/V for each captured layer and save PNGs + overlays
def reshape_for_heads(t, num_heads):
    b,s,e = t.shape
    head_dim = e // num_heads
    return t.view(b, s, num_heads, head_dim).permute(0,2,1,3).contiguous()

saved = []
for k, x_cpu in inputs_store.items():
    li = int(k.split("_")[1])
    x = x_cpu.to(device)
    seq_len, batch, E = x.shape
    att_mod = getattr(encoder_layers[li], "self_attention", None)
    if att_mod is None:
        print("No self_attention on layer", li); continue
    in_proj_w = att_mod.in_proj_weight.to(device); in_proj_b = att_mod.in_proj_bias.to(device)
    embed_dim = att_mod.embed_dim; num_heads = att_mod.num_heads
    x_b = x.permute(1,0,2).contiguous()
    proj = F.linear(x_b, in_proj_w, in_proj_b)
    q,k_,v = proj.split(embed_dim, dim=-1)
    qh = reshape_for_heads(q, num_heads); kh = reshape_for_heads(k_, num_heads)
    logits = torch.matmul(qh, kh.transpose(-2,-1)) / math.sqrt(embed_dim // num_heads)
    attn = torch.softmax(logits, dim=-1).detach().cpu().numpy()  # (B, H, N, N)

    # save raw
    rawp = OUT_DIR / f"layer_{li}_attn_raw_computed.npy"
    np.save(rawp, attn); saved.append(str(rawp))

    # avg and png
    arr_heads = attn[0] if attn.ndim==4 else attn
    H,N,_ = arr_heads.shape
    patches = N-1
    if patches > 0:
        s = int(math.sqrt(patches))
        if s*s == patches:
            avg = arr_heads.mean(axis=0)
            cls_avg = avg[0,1:]
            cls_avg = cls_avg - cls_avg.min()
            if cls_avg.max() > 0: cls_avg = cls_avg/cls_avg.max()
            grid = cls_avg.reshape(s,s)
            pngp = ATT_DIR / f"layer_{li}_attn_avg_computed.png"
            plt.figure(figsize=(4,4)); plt.imshow(grid, cmap="inferno"); plt.axis("off")
            plt.savefig(pngp, bbox_inches="tight", pad_inches=0); plt.close()
            saved.append(str(pngp))

            # overlay with existing patch grid if available
            patch_png = OUT_DIR / f"layer_{li}_patch_grid.png"
            if patch_png.exists():
                # create overlay
                att_arr = (grid - grid.min())
                if att_arr.max()>0: att_arr = att_arr/att_arr.max()
                att_img = Image.fromarray((plt.get_cmap("inferno")(att_arr)[:,:,:3]*255).astype('uint8')).convert("RGBA")
                alpha = Image.fromarray((att_arr*255).astype('uint8')).resize(att_img.size, Image.BILINEAR)
                att_img.putalpha(alpha)
                patch_img = Image.open(patch_png).convert("L")
                patch_rgb = Image.fromarray((plt.get_cmap("viridis")((np.array(patch_img).astype(float)-patch_img.min())/(patch_img.max()-patch_img.min()+1e-9))[:,:,:3]*255).astype('uint8')).convert("RGBA")
                composite = Image.alpha_composite(patch_rgb, att_img)
                draw = ImageDraw.Draw(composite)
                try:
                    font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", 18)
                except Exception:
                    font = None
                draw.rectangle([0,0, composite.size[0], 34], fill=(0,0,0,160))
                draw.text((8,6), f"Layer {li} — Attention avg overlay", fill=(255,255,255,255), font=font)
                overlayp = OVER_DIR / f"layer_{li}_overlay.png"
                composite.save(overlayp)
                saved.append(str(overlayp))
        else:
            print(f"layer {li}: patches={patches} not square; skipping png")
    else:
        print(f"layer {li}: no patch tokens (N={N}) - skipping png")

# Show results
print("Saved files:", saved)
print("\nNow displaying overlays (if created):")
from IPython.display import Image, display
for p in sorted(os.listdir(OVER_DIR)):
    if p.endswith(".png"):
        print("Showing:", p)
        display(Image(filename=str(OVER_DIR / p)))

# End of cell


Imported create_dino_hybrid_model via package import.
Model ready on cpu
Encoder layers detected: 12


RuntimeError: Model has 12 layers; none of [13, 14, 15] are valid. Choose indices < 12.

In [9]:
# Robust import cell (run once)
import os, sys, importlib.util, traceback
PROJECT_DIR = "/content/Melanoma-Dermoscopic-Prognosis"
if PROJECT_DIR not in sys.path:
    sys.path.insert(0, PROJECT_DIR)

try:
    from src.models import create_dino_hybrid_model
    print("Imported factory via package import.")
except Exception:
    print("Package import failed; falling back to file import.")
    fallback = os.path.join(PROJECT_DIR, "src", "models", "dino_hybrid.py")
    spec = importlib.util.spec_from_file_location("dino_hybrid_mod", fallback)
    mod = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(mod)
    create_dino_hybrid_model = getattr(mod, "create_dino_hybrid_model")
    print("Loaded factory from:", fallback)

# Create model (no pretrained to avoid downloads; set pretrained=True if you want)
model = create_dino_hybrid_model(
    task="classification",
    multitask=False,
    arch="vit_b_16",
    pretrained=False,
    dino_checkpoint=None,
    use_tokens=False,
    hidden_dim=256,
    dropout=0.1,
    freeze_backbone_layers=0
)
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print("Model created on", device)
# show encoder layer count (where they live may vary)
backbone = model.backbone.vit if hasattr(model, "backbone") and hasattr(model.backbone, "vit") else (model.vit if hasattr(model, "vit") else getattr(model.backbone, "vit", None))
if backbone is None:
    raise RuntimeError("Couldn't find backbone.vit on the model.")
encoder_layers = backbone.encoder.layers if hasattr(backbone, "encoder") and hasattr(backbone.encoder, "layers") else (backbone.blocks if hasattr(backbone, "blocks") else None)
print("Encoder layers container found:", type(encoder_layers), "count =", len(encoder_layers))


Imported factory via package import.
Model created on cpu
Encoder layers container found: <class 'torch.nn.modules.container.Sequential'> count = 12


In [10]:
# Append 3 encoder blocks to the ViT backbone
import copy, torch, torch.nn as nn

# encoder_layers must exist from previous cell
num_existing = len(encoder_layers)
print("Existing encoder layers:", num_existing)

# pick a source block to clone (use last block)
src_idx = num_existing - 1
src_block = encoder_layers[src_idx]
print("Cloning block at index", src_idx, "type:", type(src_block))

# create 3 new blocks by deepcopy
new_blocks = []
for i in range(3):
    b = copy.deepcopy(src_block)
    # Re-initialize key submodules to avoid copying optimizer states or bn params
    # For standard Transformer blocks, re-init linear layers and LayerNorms.
    for mod in b.modules():
        # reset linear layers' weights & biases and LayerNorm weights/biases
        if isinstance(mod, nn.Linear):
            nn.init.xavier_uniform_(mod.weight)
            if mod.bias is not None:
                nn.init.zeros_(mod.bias)
        elif isinstance(mod, nn.LayerNorm):
            if hasattr(mod, "weight"):
                nn.init.ones_(mod.weight)
            if hasattr(mod, "bias"):
                nn.init.zeros_(mod.bias)
    new_blocks.append(b)

# Append to encoder_layers ModuleList (works for ModuleList or list-like)
try:
    # If ModuleList
    for nb in new_blocks:
        encoder_layers.append(nb)
except Exception:
    # If plain list, reassign
    encoder_layers = list(encoder_layers) + new_blocks
    # patch it back onto the backbone (if attribute name is 'encoder.layers' or 'blocks')
    if hasattr(backbone, "encoder") and hasattr(backbone.encoder, "layers"):
        backbone.encoder.layers = nn.ModuleList(encoder_layers)
    elif hasattr(backbone, "blocks"):
        backbone.blocks = nn.ModuleList(encoder_layers)
    else:
        raise RuntimeError("Couldn't set encoder layers back on backbone")

print("New encoder layer count:", len(encoder_layers))


Existing encoder layers: 12
Cloning block at index 11 type: <class 'torchvision.models.vision_transformer.EncoderBlock'>
New encoder layer count: 15


In [11]:
# Example: freeze first 9 layers, fine-tune last 6 (including newly added)
freeze_until = 9  # freeze layers indices 0..8
for i, layer in enumerate(encoder_layers):
    requires = False if i < freeze_until else True
    for p in layer.parameters():
        p.requires_grad = requires
print("Parameters frozen for layers < ", freeze_until)
# Also ensure classification head params are trainable:
for p in model.parameters():
    if p.requires_grad:
        pass
# Print number of trainable params
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())
print(f"Trainable params: {trainable:,} / {total:,}")


Parameters frozen for layers <  9
Trainable params: 44,303,594 / 108,094,442


In [13]:
# Save model state, export layer summary, zip outputs and expose download links
import os, sys, zipfile
from pathlib import Path
import torch
from IPython.display import FileLink, display
import json

OUT_DIR = Path("/content/feature_outputs")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# 1) Save model.state_dict (model should already be in memory from your previous cells)
MODEL_OUT = OUT_DIR / "model_with_extra_layers.pt"
try:
    # prefer full state_dict to be safe (works even if model on CPU/GPU)
    torch.save(model.state_dict(), str(MODEL_OUT))
    print("Saved model state_dict to:", MODEL_OUT)
except Exception as e:
    print("Failed to save model state_dict:", e)
    # try saving the entire model object as fallback
    try:
        torch.save(model, str(MODEL_OUT.with_suffix(".fullmodel.pt")))
        print("Saved full model object as fallback:", MODEL_OUT.with_suffix(".fullmodel.pt"))
    except Exception as e2:
        print("Failed to save full model object:", e2)

# 2) Write a layer summary file
summary_path = OUT_DIR / "encoder_layers_summary.txt"
try:
    # attempt to inspect encoder_layers variable
    enc = encoder_layers  # should exist from prior steps
    with open(summary_path, "w") as f:
        f.write(f"Encoder layer count: {len(enc)}\n\n")
        for i, layer in enumerate(enc):
            f.write(f"Layer {i}: type={type(layer)}\n")
            # list first-level submodules for quick glance
            children = list(layer.named_children())
            if children:
                f.write("  submodules:\n")
                for name, mod in children:
                    f.write(f"    - {name}: {type(mod)}\n")
            f.write("\n")
    print("Wrote encoder layer summary to:", summary_path)
except Exception as e:
    print("Failed to write layer summary (encoder_layers may not be in scope):", e)

# 3) Optionally include your original uploaded zip in the bundle if present
orig_zip = Path("/mnt/data/Melanoma-Dermoscopic-Prognosis-main.zip")
include_orig = orig_zip.exists()
if include_orig:
    print("Found original uploaded zip at:", orig_zip)
else:
    print("Original uploaded zip not found at /mnt/data/Melanoma-Dermoscopic-Prognosis-main.zip (that's okay).")

# 4) Create a zip bundle of outputs
zip_path = OUT_DIR / "exported_model_and_summary.zip"
with zipfile.ZipFile(zip_path, "w", compression=zipfile.ZIP_DEFLATED) as zf:
    if MODEL_OUT.exists():
        zf.write(MODEL_OUT, MODEL_OUT.name)
    if summary_path.exists():
        zf.write(summary_path, summary_path.name)
    # include patch grid images if present (helpful)
    for p in OUT_DIR.glob("layer_*_patch_grid.png"):
        zf.write(p, p.name)
    # include attention raw/avg if present
    for p in OUT_DIR.glob("layer_*attn*.*"):
        zf.write(p, p.name)
    if include_orig:
        zf.write(orig_zip, orig_zip.name)

print("Created zip:", zip_path, " (size: {:.2f} MB)".format(zip_path.stat().st_size / (1024*1024)))

# 5) Expose direct download links in Colab UI
print("\nLocal files you can download:")
print(" - Model state_dict:", MODEL_OUT)
print(" - Layer summary:", summary_path)
print(" - Zip bundle:", zip_path)

# FileLink (clickable in notebook)
print("\nClick the links below to preview/download (Colab will open a preview):")
display(FileLink(str(MODEL_OUT)))
display(FileLink(str(summary_path)))
display(FileLink(str(zip_path)))

# 6) Also offer browser download using google.colab.files (works for small/medium files)
try:
    from google.colab import files
    print("\nIf you prefer immediate browser download, call files.download on the zip now.")
    # Uncomment the next line to trigger a direct browser download immediately:
    # files.download(str(zip_path))
except Exception:
    pass

# 7) Print the exact local paths (useful if you want to refer to them elsewhere)
print("\nExact local paths:")
print("MODEL_OUT =", MODEL_OUT)
print("SUMMARY   =", summary_path)
print("ZIP_FILE  =", zip_path)


Saved model state_dict to: /content/feature_outputs/model_with_extra_layers.pt
Wrote encoder layer summary to: /content/feature_outputs/encoder_layers_summary.txt
Original uploaded zip not found at /mnt/data/Melanoma-Dermoscopic-Prognosis-main.zip (that's okay).
Created zip: /content/feature_outputs/exported_model_and_summary.zip  (size: 378.21 MB)

Local files you can download:
 - Model state_dict: /content/feature_outputs/model_with_extra_layers.pt
 - Layer summary: /content/feature_outputs/encoder_layers_summary.txt
 - Zip bundle: /content/feature_outputs/exported_model_and_summary.zip

Click the links below to preview/download (Colab will open a preview):


/content/feature_outputs/model_with_extra_layers.pt

/content/feature_outputs/encoder_layers_summary.txt

/content/feature_outputs/exported_model_and_summary.zip


If you prefer immediate browser download, call files.download on the zip now.

Exact local paths:
MODEL_OUT = /content/feature_outputs/model_with_extra_layers.pt
SUMMARY   = /content/feature_outputs/encoder_layers_summary.txt
ZIP_FILE  = /content/feature_outputs/exported_model_and_summary.zip


## Optional: Save Results to Google Drive

After training, you can save the outputs to your Google Drive for later use.
